In [1]:
import numpy as np
import pandas as pd

In [2]:
# 每个参数的信息可以参考command_line
class QLearningTable:
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = actions
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)
    
    def check_state_exist(self, state):
        # self.q_table.index self.q_table的name列表
        if state not in self.q_table.index:
            self.q_table = self.q_table.append(
                pd.Series(
                    [0] * len(self.actions),
                    index=self.q_table.columns,
                    name=state
                )
            )
    
    def choose_action(self, observation):
        # q_table 表中不存在就添加到表中
        self.check_state_exist(observation)
        # action selection
        if np.random.uniform() < self.epsilon:
            # 提取要操作的表中的某一行
            state_action = self.q_table.loc[observation, :]
            # 获取数据最大的那列的名字
            action = np.random.choice(
                state_action[state_action == np.max(state_action)].index
            )
        else:
            action = np.random.choice(self.actions)
        return action
    
    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()
        else:
            q_target = r
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)
    
    def get_q_table(self):
        return self.q_table

In [3]:
from maze_env import Maze
def update(env, RL):
    def __update():
        for episode in range(100):
            # initial observation
            observation = env.reset()
            done = False
            while not done:
                # fresh env
                env.render()
                
                # RL choose action based on observation
                action = RL.choose_action(str(observation))
                
                # RL take action and get next observation and reward
                observation_, reward, done = env.step(action)
                
                # RL learn from this transition
                RL.learn(str(observation), action, reward, str(observation_))
                
                # swap observation
                observation = observation_
        print(RL.get_q_table())
        # end of game
        print('game over')
        env.destroy()
    return __update

In [4]:
env = Maze()
RL = QLearningTable(actions=list(range(env.n_actions)))
env.after(100, update(env, RL))
env.mainloop()

                                         0             1         2  \
[5.0, 5.0, 35.0, 35.0]        2.140944e-08  0.000000e+00  0.000191   
[5.0, 45.0, 35.0, 75.0]       4.092930e-08  0.000000e+00  0.000000   
[45.0, 45.0, 75.0, 75.0]      2.660600e-05 -1.000000e-02 -0.010000   
[5.0, 85.0, 35.0, 115.0]      0.000000e+00  0.000000e+00 -0.019900   
terminal                      0.000000e+00  0.000000e+00  0.000000   
[45.0, 5.0, 75.0, 35.0]       8.504021e-07  4.326395e-07  0.001478   
[85.0, 5.0, 115.0, 35.0]      0.000000e+00 -2.970100e-02  0.008865   
[125.0, 5.0, 155.0, 35.0]     3.913328e-04  4.574411e-02  0.000231   
[125.0, 45.0, 155.0, 75.0]    2.503339e-05  1.863033e-01  0.001026   
[125.0, 85.0, 155.0, 115.0]   2.178963e-05  0.000000e+00  0.006261   
[5.0, 125.0, 35.0, 155.0]     0.000000e+00  0.000000e+00  0.000000   
[45.0, 125.0, 75.0, 155.0]   -1.000000e-02  0.000000e+00  0.000090   
[125.0, 125.0, 155.0, 155.0]  0.000000e+00  0.000000e+00  0.000000   
[85.0, 125.0, 115.0,